In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import sys
import os
import gc


from tqdm.notebook import tqdm
from itertools import cycle

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import RidgeCV
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb

import shap

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 42

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

In [ ]:
%%time
train = pd.read_csv("/kaggle/input/pca-mutualinfo/train_pca.csv")
test = pd.read_csv("/kaggle/input/pca-mutualinfo/test_pca.csv")
train_labels = pd.read_csv("/kaggle/input/music-genre-datascience/music30s_trainlabel(1).csv")

train.shape, test.shape, train_labels.shape

In [ ]:
train_labels = train_labels.iloc[:,1:]

In [ ]:
train_labels.head(3)

In [ ]:
display(train.head())
display(test.head())

In [ ]:
cols = [c for c in train.columns]

In [ ]:
display(train.head(3))
display(test.head(3))

In [ ]:
train = pd.concat([train,train_labels],axis=1)

In [ ]:
target_column = 'label'

In [ ]:
features = [c for c in train.columns if c not in [target_column]]

In [ ]:
X = train.drop('label',axis=1)
y=train['label']

In [ ]:
lgb_params = {'max_depth': 11, 
              'n_estimators': 20000, 
              'learning_rate': 0.09541305857176033,
              'subsample': 0.5, 'colsample_bytree': 0.4, 
              'min_child_weight': 0.00011932256358460893, 
              'reg_lambda': 17.601337020796727, 
              'reg_alpha': 0.010231187481690964,
              'objective': 'multiclass', 
              'min_child_samples': 15, 
              'subsample_freq': 20,
             'num_class':10}

In [ ]:
X.shape, train.shape, y.shape, test.shape

In [ ]:
train['label'].head()

In [ ]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [ ]:
%%time
N_SPLITS = 5
y_preds = []
acc =[]
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = lgb.LGBMClassifier(**lgb_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

#print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test))



In [ ]:
print('the mean accuracy is: {}'.format(mean(acc)))


In [ ]:
y_preds

In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
y_pred = pd.DataFrame(columns=['lgbmC_pca'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
y_pred["lgbmC_pca"]=pred

In [ ]:
y_pred

In [ ]:
y_pred.to_csv('lgbmC_pca_pred.csv', index=False)

In [ ]:
xgb_params = {'max_depth': 5, 'n_estimators': 17000,
              'learning_rate': 0.0729884348240324, 
              'subsample': 0.30000000000000004,
              'colsample_bytree': 0.4,
              'reg_lambda': 191.18744276937747, 
              'reg_alpha': 0.01182231127307455}

In [ ]:
%%time
N_SPLITS = 5
y_preds2 = []
acc = []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = xgb.XGBClassifier(**xgb_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

#print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds2.append(model.predict(test))




In [ ]:
print('the mean accuracy is: {}'.format(mean(acc)))



In [ ]:
y_pred2 = pd.DataFrame(columns=['xgbC_pca'])
pred2= []
for j in range(0,100):
    arr = [y_preds2[0][j], y_preds2[1][j], y_preds2[2][j], y_preds2[3][j],y_preds2[4][j]]
    pred2.append(mode(arr))

In [ ]:
y_pred2["xgbC_pca"]=pred2

In [ ]:
y_pred2

In [ ]:
y_pred2.to_csv('xgbC_pca_pred.csv', index=False)